# 1. 주민센터 데이터 
- 센터 이름(행정동 컬럼도 추출 가능) , 주소(위도, 경도 추출을 위해) 

In [3]:
import numpy as np
import pandas as pd 
import os
current_path = os.getcwd()

pd.set_option('display.max_rows', 5)

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

#Crawling 
import requests 
from bs4 import BeautifulSoup 
import bs4
import time

#Clustering 
from sklearn.cluster import KMeans 
from sklearn import metrics 


## 데이터 크롤링(셀레니움) - for 주민센터 좌표 

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [3]:
! pip install selenium

* 크롬드라이버를 이용해 페이지 접속 

In [4]:
#chrome창(웹드라이버) 열기 
browser = webdriver.Chrome("C:\/chromedriver.exe") 

# 홈페이지 접속하기 
url = 'https://www.myhome.go.kr/hws/portal/cmt/selectCmtSrvCntrListView.do'
browser.get(url) #접속

In [5]:
name_ = []
address_ = []

In [6]:
schTbody = browser.find_element(By.ID, 'schTbody').find_elements(By.TAG_NAME, 'tr')
schTbody

# 1 page 정보 추출 
for tr in schTbody : 
    td_list = tr.find_elements(By.TAG_NAME, 'td')
    num = td_list[0].text
    name = td_list[1].text
    address = td_list[3].find_element(By.TAG_NAME, 'a').text
    print(name, address)

청운효자동 주민센터 서울특별시 종로구 자하문로 92
사직동 주민센터 서울특별시 종로구 사직로9길 1
종로1-4가동 주민센터 서울특별시 종로구 종로17길 8
삼청동 주민센터 서울특별시 종로구 삼청로 107
가회동 주민센터 서울특별시 종로구 북촌로 35
종로5-6가동 주민센터 서울특별시 종로구 종로35가길 19
이화동 주민센터 서울특별시 종로구 이화장길 33
혜화동 주민센터 서울특별시 종로구 혜화로 12
창신1동 주민센터 서울특별시 종로구 지봉로 8-6
창신2동 주민센터 서울특별시 종로구 창신길 62


* 전체 페이지 정보 긁어오기(단, 424번까지) 

In [7]:
name_ = []
address_ = []

schTbody = browser.find_element(By.ID, 'schTbody').find_elements(By.TAG_NAME, 'tr')
page = 1
while 1 != 0: #계속 돌리기 위함 
    for tr in schTbody : 
        td_list = tr.find_elements(By.TAG_NAME, 'td')
        name = td_list[1].text
        address = td_list[3].find_element(By.TAG_NAME, 'a').text
        name_.append(name)
        address_.append(address)
        if td_list[0].text == 425 :
            break
    page +=10
    browser.execute_script(f'javascript:fnCmtSrvvCntrList("{page}")') # "다음"page 클릭을 위한 코드
    time.sleep(1)
    schTbody = browser.find_element(By.ID, 'schTbody').find_elements(By.TAG_NAME, 'tr')


print(len(name_))
#  driver.execute_script 함수 (자바스크립트로 아이디, 패스워드를 넘겨주는 형태)

KeyboardInterrupt: 

* 흐음..전체 지역에서 424번까지 돌릴려했으나... ^^ 결국 실패.
* Plan B를 선택 (지역을 "서울"로 한정지은 다음 정보 가져오기) 

In [22]:
# 지역선택 박스 클릭
local = browser.find_element(By.ID, 'srcBrtCd')
local.click()

#서울지역 클릭하기 
option = local.find_elements(By.CSS_SELECTOR, 'option')
option[1].click() #서울클릭

In [23]:
name_ = []
address_ = []

schTbody = browser.find_element(By.ID, 'schTbody').find_elements(By.TAG_NAME, 'tr')
page = 1
while len(schTbody) != 0:
    for tr in schTbody : 
        td_list = tr.find_elements(By.TAG_NAME, 'td')
        name = td_list[1].text
        address = td_list[3].find_element(By.TAG_NAME, 'a').text
        name_.append(name)
        address_.append(address)

    page +=1 #page 카운트 
    browser.execute_script(f'javascript:fnCmtSrvvCntrList("{page}")') # 다음페이지 클릭
    time.sleep(1)
    schTbody = browser.find_element(By.ID, 'schTbody').find_elements(By.TAG_NAME, 'tr') #테이블 업뎃


print(len(name_))
#  driver.execute_script 함수 (자바스크립트로 아이디, 패스워드를 넘겨주는 형태)

424


In [24]:
columns = ['주민센터명', '주소']
df = pd.DataFrame(zip(name_, address_), columns = columns) # 빈데이터 프레임 만들기 
df

,주민센터명,주소
0,청운효자동 주민센터,서울특별시 종로구 자하문로 92
1,사직동 주민센터,서울특별시 종로구 사직로9길 1
...,...,...
422,천호3동 주민센터,서울특별시 강동구 진황도로23길 7
423,강일동 주민센터,서울특별시 강동구 아리수로93길 9-14


[소감]

* 이름이랑 주소 가져오겠다고 ... 뷰티풀숲부터 셀레니움 뒤적거린 시간을 생각하니 눈물이 나네요 ^^ 중간에 포기할까 했지만 결국 해낸 내 자신 칭찬해! 

[참고사이트]
* 지역선택 : (https://brunch.co.kr/@jk-lab/18)
* 센터크롤링 참고 : https://datalabbit.tistory.com/45 , https://steadiness-193.tistory.com/118 , https://book.coalastudy.com/data_crawling/week6/stage4 , 
https://suriisurii.tistory.com/62

## 데이터 크롤링(셀레니움) - for 복지시설 주소

In [125]:
#chrome창(웹드라이버) 열기 
browser = webdriver.Chrome("C:\/chromedriver.exe") 

# 홈페이지 접속하기 
url = 'https://wis.seoul.go.kr/hope/customizedSearch.do'
browser.get(url) #접속

In [126]:
# 노인복지관 클릭
local = browser.find_element(By.ID, 'tab1s12')
local.click()

In [127]:
# 리스트 만들기 
names = []
address_ = []

In [128]:
schTbody = browser.find_element(By.ID, 'content').find_elements(By.TAG_NAME, 'dl')
page = 1
while len(schTbody) != 0:
    for dl in schTbody : 
        dt_list = dl.find_element(By.TAG_NAME, 'dt')
        ul_list = dl.find_element(By.TAG_NAME, 'ul')
        name = dt_list.find_element(By.TAG_NAME, 'a').text
        address = ul_list.find_element(By.TAG_NAME, 'li').text
        names.append(name)
        address_.append(address)

    page +=1 #page 카운트 
    browser.execute_script(f'javascript:pagedIng("{page}")') # 다음페이지 클릭
    time.sleep(3)
    schTbody = browser.find_element(By.ID, 'content').find_elements(By.TAG_NAME, 'dl') #테이블 업뎃


print(len(names))


86


In [129]:
columns = ['시설명', '주소']
df = pd.DataFrame(zip(names, address_), columns = columns) # 빈데이터 프레임 만들기 
df

,시설명,주소
0,마포노인종합복지관,주소 서울특별시 마포구 서강로 68
1,강남구립강남시니어플라자,주소 서울특별시 강남구 봉은사로 332
...,...,...
84,방배느티나무쉼터,주소 서울특별시 서초구 남부순환로287길
85,용답동 노인복지센터,주소 서울특별시 성동구 용답동


In [130]:
df['주소'] = df['주소'].str[3:]
df

,시설명,주소
0,마포노인종합복지관,서울특별시 마포구 서강로 68
1,강남구립강남시니어플라자,서울특별시 강남구 봉은사로 332
...,...,...
84,방배느티나무쉼터,서울특별시 서초구 남부순환로287길
85,용답동 노인복지센터,서울특별시 성동구 용답동


In [108]:
df.to_excel('복지관주소.xlsx')

## 주소를 이용해 위도, 경도 가져오기

* 지오코딩(주소를 통해 위도, 경도 좌표 얻기) 

In [81]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [82]:
from geopy.geocoders import Nominatim

In [83]:
geolocoder = Nominatim(user_agent = 'South Korea', timeout = None)

def geocoding(address) : 
    try : 
        geo= geolocoder.geocode(address)
        crd = (str(geo.latitude), str(geo.longitude))
        return crd
    
    except : 
        return (0,0)

In [109]:
lats = []
logs = []

for address in df['주소'] : 
    crd = geocoding(address)
    lats.append(crd[0])
    logs.append(crd[1])

lats = pd.Series(lats)
logs = pd.Series(logs)

In [110]:
df = pd.concat([df, lats], axis = 1)
df = pd.concat([df, logs], axis = 1)
df

,복지관명,주소,위도,경도,0,0
0,마포노인종합복지관,서울특별시 마포구 서강로 68,37.5549534,126.9366361,37.5549534,126.9366361
1,강남구립강남시니어플라자,서울특별시 강남구 봉은사로 332,37.5549534,126.9366361,37.5135682,127.05643
...,...,...,...,...,...,...
84,방배느티나무쉼터,서울특별시 서초구 남부순환로287길,37.4773892,126.9877348,37.4773892,126.9877348
85,용답동 노인복지센터,서울특별시 성동구 용답동,37.56211,127.05467,37.56211,127.05467


In [112]:
df.drop(columns = ['위도', '경도'], inplace = True)

In [113]:
df.columns = ['복지관명', '주소', '위도', '경도']
df

,복지관명,주소,위도,경도
0,마포노인종합복지관,서울특별시 마포구 서강로 68,37.5549534,126.9366361
1,강남구립강남시니어플라자,서울특별시 강남구 봉은사로 332,37.5135682,127.05643
...,...,...,...,...
84,방배느티나무쉼터,서울특별시 서초구 남부순환로287길,37.4773892,126.9877348
85,용답동 노인복지센터,서울특별시 성동구 용답동,37.56211,127.05467


In [114]:
# 오류난 곳 주소 변경해서 다시하기 
error = df[df.위도 == 0]
error.head(5)

,복지관명,주소,위도,경도
33,방학동어르신복지관,서울특별시 도봉구 시루봉로 15라길 59-9,0,0
38,도봉동어르신복지관,서울특별시 도봉구 도봉로 169가길 52,0,0
64,강서구립곰달래어르신복지센터,서울특별시 강서구 강서로5길 50,0,0


In [93]:
# 복지관 error 주소변경 
df.iloc[27, 1] = '서울특별시 구로구 부일로9길 111'
df.iloc[33, 1] = '서울특별시 도봉구 시루봉로 15라길 59-9'
df.iloc[38,1] = '서울특별시 도봉구 도봉로 169가길 52'
df.iloc[62, 1] = '서울특별시 노원구 노원로16길 15'
df.iloc[64, 1] = '서울특별시 강서구 강서로5길 50'
df.iloc[71,1] = '서울특별시 중랑구 망우로65길 20-0'

In [99]:
# df에서 위도 경도 채워주기 
error_ad = error.주소
error_ad

27         서울특별시 구로구 부일로9길 111
33    서울특별시 도봉구 시루봉로 15라길 59-9
                ...           
64          서울특별시 강서구 강서로5길 50
71       서울특별시 중랑구 망우로65길 20-0
Name: 주소, Length: 6, dtype: object

In [100]:
for i in range(6) :
    df.iloc[error_ad.index[i],2] = geocoding(error_ad.values[0])[0]
    df.iloc[error_ad.index[i],3] = geocoding(error_ad.values[0])[1]
    
df[df.위도 == 0]

,복지관명,주소,위도,경도


In [101]:
df

,복지관명,주소,위도,경도
0,마포노인종합복지관,서울특별시 마포구 서강로 68,37.5549534,126.9366361
1,강남구립강남시니어플라자,서울특별시 강남구 봉은사로 332,37.5549534,126.9366361
...,...,...,...,...
84,방배느티나무쉼터,서울특별시 서초구 남부순환로287길,37.4773892,126.9877348
85,용답동 노인복지센터,서울특별시 성동구 용답동,37.56211,127.05467


In [116]:
df.to_excel('복지관명.xlsx')

In [117]:
df.to_csv('복지관명_위도경도.csv')

----------------------------------

In [30]:
# 주민센터주소변경 
df.iloc[115, 1] = '서울 성북구 고려대로 16'
df.iloc[238, 1] = '서울 양천구 목동서로 353'
df.iloc[274,1] = '서울 구로구 오리로15길 32'

In [108]:
# df에서 위도, 경도 채워주기  
error_ad = error.주소
error_ad

for i in range(3) : 
    df.iloc[error_ad.index[i], 2] = geocoding(error_ad.values[0])[0]
    df.iloc[error_ad.index[i], 3] = geocoding(error_ad.values[0])[1]

df[df.위도 == 0]

,주민센터명,주소,위도,경도


In [109]:
df

,주민센터명,주소,위도,경도
0,청운효자동 주민센터,서울특별시 종로구 자하문로 92,37.584083,126.970636
1,사직동 주민센터,서울특별시 종로구 사직로9길 1,37.576191,126.968829
...,...,...,...,...
422,천호3동 주민센터,서울특별시 강동구 진황도로23길 7,37.539519,127.129328
423,강일동 주민센터,서울특별시 강동구 아리수로93길 9-14,37.564942,127.173915


In [110]:
df.to_csv('jmst_lat_log', index = False)

# 위도경도로 주변시설 개수 찾기

In [38]:
!pip install haversine

from haversine import haversine

In [130]:
df = pd.read_csv('jmst_lat_log')
df.head(1)

,주민센터명,주소,위도,경도
0,청운효자동 주민센터,서울특별시 종로구 자하문로 92,37.584083,126.970636


In [131]:
df_bus = pd.read_csv('data/서울시버스정류소좌표데이터(2022.04.15).csv',
                    encoding = 'cp949')
df_bus

,ARS-ID,정류소명,X좌표,Y좌표
0,1001,종로2가사거리,126.987750,37.569765
1,1002,창경궁.서울대학교병원,126.996566,37.579183
...,...,...,...,...
11313,25998,성내시장,127.125497,37.536155
11314,25999,천호우체국.로데오거리,127.127337,37.540343


In [132]:
#df_bus (경도 위도)를 위도 경도로  바꿔준다 
df_bus[['ARS-ID', '정류소명', 'Y좌표', 'X좌표']]

,ARS-ID,정류소명,Y좌표,X좌표
0,1001,종로2가사거리,37.569765,126.987750
1,1002,창경궁.서울대학교병원,37.579183,126.996566
...,...,...,...,...
11313,25998,성내시장,37.536155,127.125497
11314,25999,천호우체국.로데오거리,37.540343,127.127337


In [133]:
#df의 경도, 위도가 string으로 되어있었따 
df['경도'] = pd.to_numeric(df['경도'])
df['위도'] = pd.to_numeric(df['위도'])
df

,주민센터명,주소,위도,경도
0,청운효자동 주민센터,서울특별시 종로구 자하문로 92,37.584083,126.970636
1,사직동 주민센터,서울특별시 종로구 사직로9길 1,37.576191,126.968829
...,...,...,...,...
422,천호3동 주민센터,서울특별시 강동구 진황도로23길 7,37.539519,127.129328
423,강일동 주민센터,서울특별시 강동구 아리수로93길 9-14,37.564942,127.173915


In [123]:
df.drop(columns = '200m내버스정류장개수', inplace = Truei)

In [126]:
from tqdm import tqdm 
#tqdm : for문을 돌릴때 얼마만큼 진행되는지에 대한 피드백을 주는 기능 
## 사용법 : for 문의 in구문으로 tqdm을 감싸면 된다
# for i in tqdm(range(1, 600)) :


In [127]:
# 함수 만들기 (200m 이하 거리에 위치한 버스정류장 찾기)
def find_bus_cnt(center, store, column) :
    for i in tqdm(range(len(center))) :
        distance = []
        for j in range(len(store)) : 
            centerloc = tuple(df[['위도', '경도']].iloc[i])
            storeloc = tuple(df_bus[['Y좌표', 'X좌표']].iloc[j])
            distance.append(haversine(centerloc, storeloc, unit = 'm'))
            
        distance = [d for d in distance if d <= 200.0]
        center[column][i] = len(distance)
        
    return center


In [128]:
df['200m내버스정류장개수'] = 0 
df = find_bus_cnt(df, df_bus,'200m내버스정류장개수' )
df

  0%|                                                                                          | 0/424 [00:00<?, ?it/s]C:\Users\debbie\AppData\Local\Temp/ipykernel_8820/1259571478.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  center[column][i] = len(distance)
100%|█████████████████████████████████████████████████████████████████████████████| 424/424 [10:10:27<00:00, 86.38s/it]


,주민센터명,주소,위도,경도,200m내버스정류장개수
0,청운효자동 주민센터,서울특별시 종로구 자하문로 92,37.584083,126.970636,4
1,사직동 주민센터,서울특별시 종로구 사직로9길 1,37.576191,126.968829,7
...,...,...,...,...,...
422,천호3동 주민센터,서울특별시 강동구 진황도로23길 7,37.539519,127.129328,4
423,강일동 주민센터,서울특별시 강동구 아리수로93길 9-14,37.564942,127.173915,11


In [134]:
df

,주민센터명,주소,위도,경도
0,청운효자동 주민센터,서울특별시 종로구 자하문로 92,37.584083,126.970636
1,사직동 주민센터,서울특별시 종로구 사직로9길 1,37.576191,126.968829
...,...,...,...,...
422,천호3동 주민센터,서울특별시 강동구 진황도로23길 7,37.539519,127.129328
423,강일동 주민센터,서울특별시 강동구 아리수로93길 9-14,37.564942,127.173915
